In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import resample
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.utils import resample

from joblib import Parallel, delayed
from IPython import embed

from src.utils import make_partitions

In [2]:
import numpy as np
import pandas as pd
import importlib
import tqdm
import sys

In [3]:
from src.utils import format_data
import src.modeling as tools
sys.path.append('./configs')

In [28]:
importlib.reload(tools)

<module 'src.modeling' from '/home/gbarchi/Documentos/Trust/OCEAN-TRUST/src/modeling.py'>

In [5]:
configs=importlib.import_module('experiment_14')

In [6]:
feat_df=pd.read_csv('data/features/new_partitions-egemaps_all_audio.csv')
labels_df=pd.read_csv('data/labels/stratified_df.csv')
filter='data/lists/all_audio_complete_set.txt'

In [7]:
df=format_data(feat_df,labels_df,filter)

In [8]:
df_train=df.sample(n=8000,replace=True,random_state=42)
df_val=df[~df['basename'].isin(df_train['basename'])]
feature_tags=configs.feature_tags
label_tags=configs.label_tags
seed=configs.seed
RF_reg= tools.train_model (df_train,feature_tags,label_tags,seed)

In [15]:
r2_all,MAE_all,MSE_all,RMSE_all,y_val,predictions= tools.predict(RF_reg,df_val,feature_tags,label_tags)

In [30]:
df_importance=tools.top_feature_importance(RF_reg.feature_importances_,configs.feature_tags,10)

In [45]:
a=pd.read_csv('/home/gbarchi/Documentos/Trust/OCEAN-TRUST/results/importance_1/results_importance.csv',index_col=0)

In [72]:
a

,importance,feature_tag,fold,seed,filter,feature
0,0.107674,speech_ratio,0,0,no_music_list_manual_annot,new_partitions-egemaps_silero_speech
1,0.055706,F0semitoneFrom27.5Hz_sma3nz_amean,0,0,no_music_list_manual_annot,new_partitions-egemaps_silero_speech
2,0.024361,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,0,0,no_music_list_manual_annot,new_partitions-egemaps_silero_speech
3,0.021341,mfcc2V_sma3nz_amean,0,0,no_music_list_manual_annot,new_partitions-egemaps_silero_speech
4,0.019703,F2frequency_sma3nz_stddevNorm,0,0,no_music_list_manual_annot,new_partitions-egemaps_silero_speech
...,...,...,...,...,...,...
5,0.024196,mfcc2V_sma3nz_amean,4,9,no_music_list_manual_annot,new_partitions-egemaps_silero_speech
6,0.023500,loudness_sma3_stddevRisingSlope,4,9,no_music_list_manual_annot,new_partitions-egemaps_silero_speech
7,0.015243,F2frequency_sma3nz_stddevNorm,4,9,no_music_list_manual_annot,new_partitions-egemaps_silero_speech
8,0.014565,mfcc2_sma3_amean,4,9,no_music_list_manual_annot,new_partitions-egemaps_silero_speech


In [74]:
pd.DataFrame(a.groupby('feature_tag')['importance'].mean()).sort_values('importance',ascending=False).head(5)

,importance
feature_tag,
speech_ratio,0.107392
F0semitoneFrom27.5Hz_sma3nz_amean,0.038186
F0semitoneFrom27.5Hz_sma3nz_percentile80.0,0.026062
F0semitoneFrom27.5Hz_sma3nz_percentile20.0,0.025033
mfcc2V_sma3nz_amean,0.023989
